In [ ]:
%%capture
# Installs Unsloth, Xformers (for speed), and TRL (for training loops)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Supports RoPE Scaling internally
dtype = None # None = auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# Download the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from google.colab import files
from datasets import load_dataset

# 1. Upload the file
print("👇 PLEASE UPLOAD YOUR 'd.txt' FILE NOW 👇")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 2. Load the dataset
dataset = load_dataset("json", data_files = filename, split = "train")

# 3. Define the Prompt Format
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN or it goes on forever!

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)
print("Data Loaded and Formatted Successfully!")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # <-- CHANGE THIS to 300 for a longer run!
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
print("STARTING TRAINING... ")
trainer_stats = trainer.train()

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# A Test Question (Change this string to test different things)
my_question = "If a shopkeeper sells a pen for $20 with a 10% profit, what is the cost price?"

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Solve the following math problem in Hinglish explicitly showing your steps.", # Instruction
        my_question, # Input
        "", # Output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# A Test Question (Change this string to test different things)
my_question = "If a shopkeeper sells a pen for $20 with a 10% profit, what is the cost price?"

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Solve the following math problem in Hinglish explicitly showing your steps.", # Instruction
        my_question, # Input
        "", # Output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

In [ ]:

model.save_pretrained("/content/drive/MyDrive/Hinglish_Llama3_Model")
tokenizer.save_pretrained("/content/drive/MyDrive/Hinglish_Llama3_Model")
print("Saved to 'Hinglish_Llama3_Model' in your Drive.")